In [1]:
#1 m có thể chạy hf token của m nếu m có access viet mistral rồi
from kaggle_secrets import UserSecretsClient
from huggingface_hub import login
secret_label = "HF_TOKEN"
secret_value = UserSecretsClient().get_secret(secret_label)
login(secret_value)

In [3]:
model_name = "Viet-Mistral/Vistral-7B-Chat"

In [2]:
#2
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 14.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 34.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 30.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 19.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 10.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 31.2 MB/s eta 0:00:0000:0100:01
  Attempting uninst

In [8]:
#3
import gc
import torch

def clear_cuda():
    gc.collect()                    # Python garbage collection
    if torch.cuda.is_available():
        torch.cuda.empty_cache()    # Releases unreferenced memory back to the pool
        torch.cuda.ipc_collect()    # Cleans up inter-process memory
#clear_cuda()

In [6]:
#4
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import re
import torch

def is_7b_model(model_name): # use in desperate times
    #if model_name in  ["Qwen/Qwen3-8B", "Qwen/Qwen2-7B", "Qwen/Qwen2.5-7B-Instruct", "google/gemma-7b-it", "vilm/vinallama-7b-chat", "Viet-Mistral/Vistral-7B-Chat"]:
    #    return True
    return False

def load_model(model_name):
    print(f"Loading model: {model_name}")
    clear_cuda()
    try:
        if is_7b_model(model_name):
            print(f"Loading big boy: {model_name}")
            quant_config = BitsAndBytesConfig(
                load_in_4bit = True,
                bnb_4bit_compute_dtype=torch.float16,
                bnb_4bit_use_double_quant=True,
                bnb_4bit_quant_type="nf4",
            )
            tokenizer = AutoTokenizer.from_pretrained(
                model_name,
                trust_remote_code=True,
                model_max_length=8192
            )
            model = AutoModelForCausalLM.from_pretrained(
                model_name,
                quantization_config=quant_config,
                device_map="auto",          
                trust_remote_code=True
            )
            print("Model successfully loaded across devices:")
            for name, param in model.named_parameters():
                if param.device.type == 'cuda':
                    print(f" - {name} on {param.device}")
                    break  # print only one to verify
            prev_model = model_name
        else: 
            tokenizer = AutoTokenizer.from_pretrained(model_name)
            model = AutoModelForCausalLM.from_pretrained(
                model_name,
                torch_dtype="auto", #float16 for gemma idk why auto doesnt work
                device_map="auto",          
            )
            print("Model successfully loaded across devices:")
            for name, param in model.named_parameters():
                if param.device.type == 'cuda':
                    print(f" - {name} on {param.device}")
                    break  # print only one to verify
            prev_model = model_name
        return tokenizer, model
    except Exception as e:
        print("Error loading model:", e)
        return None, None

#model_name = model_list[0]
#tokenizer, model = load_model("Viet-Mistral/Vistral-7B-Chat")

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

def load_model(model_name):
    print(f"Loading model: {model_name}")
    clear_cuda()
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype="float16", #float16 for gemma idk why auto doesnt work
        device_map="auto",          
    )
    print("Model successfully loaded across devices:")
    for name, param in model.named_parameters():
       if param.device.type == 'cuda': 
           print(f" - {name} on {param.device}")
           break  # print only one to verify
    return tokenizer, model
#tokenizer, model = load_model(model_name)

In [ ]:
print(model.device)

In [3]:
#5
import pandas as pd
label = pd.read_csv('/kaggle/input/hs-history-2024/labeled-history-202x-dedup.csv')
label.drop('Unnamed: 0', axis=1, inplace=True)
label = label[label['dang_cau_hoi'].notna()].reset_index(drop=True)
label = label.rename(columns={'options': 'choices'})
label

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,id,answer,question_type,grade,choices,question,nhan_biet,thong_hieu,van_dung,van_dung_cao,dang_cau_hoi
0,history-2022-302-35,A,multiple_choice,12.0,"['Có sự linh hoạt, sáng tạo trong phương án tá...",Chiến dịch Biên giới thu-đông (1950) và chiến ...,NaN,NaN,1.0,NaN,"So sánh, đối chiếu"
1,history-2022-304-37,A,multiple_choice,12.0,['Sử dụng đòn tiến công chiến lược của lực lượ...,Chiến dịch Biên giới thu-đông (1950) và chiến ...,NaN,NaN,1.0,NaN,"So sánh, đối chiếu"
2,history-2020-304-21,A,multiple_choice,12.0,"['sự bất lực của quân đội Sài Gòn', 'nhiệm vụ ...",Chiến thắng Phước Long (tháng 1-1975) của quân...,NaN,NaN,1.0,NaN,Đánh giá
3,history-2020-301-32,A,multiple_choice,12.0,"['khả năng thắng lớn của quân giải phóng.', 'q...",Chiến thắng Phước Long (tháng 1-1975) của quân...,NaN,NaN,1.0,NaN,Đánh giá
4,history-2020-302-23,A,multiple_choice,12.0,"['sức mạnh to lớn của quân giải phóng.', 'quân...",Chiến thắng Phước Long (tháng 1-1975) của quân...,NaN,NaN,1.0,NaN,Đánh giá
...,...,...,...,...,...,...,...,...,...,...,...
555,history-2021-304-9,A,multiple_choice,12.0,"['Thực dân Pháp bị thiệt hại nặng nề', 'Nhật đ...","Ở Đông Dương, thực dân Pháp đề ra kế hoạch quâ...",NaN,1.0,NaN,NaN,Bối cảnh/Nguyên nhân
556,history-2021-301-11,C,multiple_choice,12.0,"['Quân phiệt Nhật Bản tiến vào Đông Dương.', '...","Ở Đông Dương, thực dân Pháp đề ra kế hoạch quâ...",NaN,1.0,NaN,NaN,Bối cảnh/Nguyên nhân
557,history-2021-302-3,B,multiple_choice,12.0,"['Quân phiệt Nhật Bản tiến vào Đông Dương.', '...","Ở Đông Dương, thực dân Pháp đề ra kế hoạch quâ...",NaN,1.0,NaN,NaN,Bối cảnh/Nguyên nhân
558,history-2021-303-2,C,multiple_choice,12.0,"['Quân Trung Hoa Dân quốc tiến vào Việt Nam.',...","Ở Đông Dương, thực dân Pháp đề ra kế hoạch quâ...",NaN,1.0,NaN,NaN,Bối cảnh/Nguyên nhân


In [ ]:
import pandas as pd
hs_test = pd.read_csv('/kaggle/input/vmlu-test-dataset/hs_test.csv')
hs_test.drop(columns=['Unnamed: 0'], inplace=True)
hs_test

In [18]:
def prompter(question, choices):
    if isinstance(choices, str):
        choices_list = "\n".join([f"{chr(65+i)}. {c}" for i,c in enumerate(ast.literal_eval(choices))])
    else:
        choices_list = "\n".join([f"{chr(65+i)}. {c}" for i,c in enumerate(choices)])
    prompt = f"""### Trả lời câu hỏi lịch sử sau đây:
{question}
### Đây là các lựa chọn:
{choices_list}
### Chọn câu trả lời đúng và đặt trong ngoặc vuông như: [A], [B], [C] hoặc [D]. Hãy nghĩ từng bước một.
Không lặp lại câu hỏi hoặc lựa chọn. Không cần giải thích lại đáp án.
Nếu bạn muốn giải thích, hãy trả lời câu hỏi bằng chữ cái [A], [B], [C] hoặc [D] trước, sau đó giải thích câu trả lời của bạn.
### Answer: """
    return prompt

In [17]:
def prompter_en(question, choices):
    if isinstance(choices, str):
        choices_list = "\n".join([f"{chr(65+i)}. {c}" for i,c in enumerate(ast.literal_eval(choices))])
    else:
        choices_list = "\n".join([f"{chr(65+i)}. {c}" for i,c in enumerate(choices)])
    prompt = f"""### Answer this history question:
{question}
### Here are your choices:
{choices_list}
### Choose the correct answer and put it inside square brackets like: [A], [B], [C], or [D]. Let's think step by step.
Do **NOT** repeat the question or choices. You do NOT need explain your answer. 
If you are going to explain yourself, answer the question with a letter [A], [B], [C] or [D] first, then explain your answer.
### Answer: """
    return prompt

In [9]:
#6
import ast
# lowered to 3 shot for viet mistral so this fucking thing will run
def is_chat_model(model_name):
    return any(x in model_name.lower() for x in ["chat", "qwen", "gemma", "llama", "vistral"])

def is_vietnamese_model(model_name):
    return any(x in model_name.lower() for x in ["vinallama", "vistral", "viet", "vi-"])

def build_fewshot_cot_prompt(model_name, question, choices):
    if isinstance(choices, str):
        choices_list = "\n".join([f"{chr(65+i)}. {c}" for i,c in enumerate(ast.literal_eval(choices))])
    else:
        choices_list = "\n".join([f"{chr(65+i)}. {c}" for i,c in enumerate(choices)])

    fewshot_cot_vi = f"""
    Bạn sẽ được cung cấp câu hỏi lịch sử cấp trung học phổ thông với các lựa chọn của câu hỏi đó.
    Suy nghĩ từng bước một và trả lời câu hỏi được cho bằng một chữ cái [A], [B], [C] hay [D]. 
    Hãy trả lời đáp án theo định dạng giống với ví dụ trước khi giải thích.
    
    ### MỤC TIÊU
    Phân tích câu hỏi, và suy nghĩ từng bước một. Đưa ra MỘT câu trả lời đúng trong bốn lựa chọn được cho và đặt chữ cái của câu trả lời trong ngoặc vuông như sau: `[A]`, `[B]`, `[C]` HOẶC `[D]`. 

    ## QUY TẮC BẮT BUỘC.
    1. **KHÔNG** được lặp lại câu hỏi hoặc các lựa chọn. Quy tắc này phải tuân thủ **NGHIÊM NGẶT**.
    2. Nếu bạn có định giải thích, hãy trả lời bằng đáp án bạn chọn trước khi giải thích. 
    3. Đưa ra MỘT đáp án cuối cùng. Đáp án này PHẢI đưa ra TRƯỚC khi bạn giải thích đáp án.
    4. Học cách trả lời và đưa ra lập luận theo các ví dụ ở dưới.
    
    ### VÍ DỤ MẪU
    
    ---

    Câu hỏi: Phong trào cách mạng 1930-1931, phong trào dân chủ 1936-1939 và phong trào giải phóng dân tộc 1939-1945 ở Việt Nam đều khẳng định trong thực tiễn
    A. Vai trò lãnh đạo quần chúng đấu tranh chính trị và vũ trang của chính đảng cách mạng
    B. Tầm quan trọng của công tác xây dựng lực lượng chính trị và căn cứ địa cách mạng
    C. Vai trò xung kích của lực lượng vũ trang trong hỗ trợ quần chúng đấu tranh chính trị
    D. Bước phát triển của cách mạng trên hai địa bàn chiến lược nông thôn và thành thị

    Answer: [A]
    Lập luận:
    - Cả ba phong trào đều thể hiện khả năng lãnh đạo đấu tranh chính trị và vũ trang của Đảng Cộng sản Việt Nam.
    - Khẳng định vai trò lãnh đạo của chính đảng cách mạng trong tổ chức quần chúng.
    => Đáp án đúng là A.
    
    ---
    
    Câu hỏi: Hoạt động của Nguyễn Ái Quốc trong những năm 1920-1930 có điểm mới nào so với hoạt động của các sĩ phu tiến bộ Việt Nam đầu thế kỉ XX?
    A. Tập hợp lực lượng cách mạng gồm nhiều tầng lớp trong xã hội
    B. Thành lập Chi bộ Cộng sản đầu tiên của Việt Nam ở Bắc Kì
    C. Trang bị lí luận cách mạng giải phóng dân tộc cho thanh niên
    D. Vận động thanh niên tham gia tổ chức yêu nước chống đế quốc

    
    Answer: [C]
    Lập luận:
    - Sĩ phu đầu thế kỉ XX chủ yếu kêu gọi lòng yêu nước, cải cách.
    - Nguyễn Ái Quốc truyền bá chủ nghĩa Mác – Lênin và đào tạo thanh niên theo lí luận cách mạng giải phóng dân tộc.
    => Đáp án đúng là C.
    
    ---
    
    Câu hỏi: Trong khởi nghĩa từng phần (từ tháng 3 đến tháng 8-1945), nhân dân Việt Nam ở Bắc Kì và Bắc Trung Kì thực hiện khẩu hiệu
    A. "Phá kho thóc, giải quyết nạn đói"
    B. "Người cày có ruộng"
    C. "Tăng gia sản xuất"
    D. "Không một tấc đất bỏ hoang"

    
    Answer: [A]
    Lập luận:
    - Thời kỳ này nạn đói ở miền Bắc rất nghiêm trọng.
    - Việt Minh phát động khẩu hiệu "Phá kho thóc, giải quyết nạn đói" để huy động quần chúng.
    => Đáp án đúng là A.
    
    ---

    Câu hỏi: Trong 5 năm đầu (1986-1990) của công cuộc đổi mới, Việt Nam đạt được một trong những thành tựu nào sau đây?
    A. Hoàn thành hiện đại hóa đất nước
    B. Kiềm chế một bước đà lạm phát
    C. Xuất khẩu gạo đứng đầu thế giới
    D. Hoàn thành công nghiệp hóa đất nước

    
    Answer: [B]
    Lập luận:
    - Giai đoạn đầu của Đổi mới tập trung ổn định kinh tế.
    - Thành tựu nổi bật là bước đầu kiềm chế được lạm phát, chưa hiện đại hóa hay công nghiệp hóa hoàn toàn.
    => Đáp án đúng là B.
    
    ---

    Câu hỏi: Trong tháng 12-1993, Hiến pháp Liên bang Nga được ban hành, quy định thể chế
    A. Quân chủ chuyên chế
    B. Quân chủ lập hiến
    C. Tổng thống Liên bang
    D. Cộng hòa xã hội chủ nghĩa

    
    Answer: [C]
    Lập luận:
    - Hiến pháp 1993 của Nga quy định thể chế cộng hòa tổng thống liên bang.
    => Đáp án đúng là C.
    
    ---
    
    Trả lời câu hỏi lịch sử sau: {question}
    
    Các phương án trả lời: {choices_list}
    
    Answer:""".strip()
    
    fewshot_cot_en = f"""
    You are a history expert with deep knowledge. 
    You will be given a high school history multiple-choice question with four options.  
    If you provide reasoning, **always give the answer first, then the explanation**.

    ### OBJECTIVE
    Analyze the question step by step. Provide ONLY ONE correct answer among the four options, and put the letter inside square brackets like: `[A]`, `[B]`, `[C]`, or `[D]`. 

    ### STRICT RULES
    1. **DO NOT** repeat the question or the options. Follow this rule strictly.  
    2. If you explain, state the answer you choose first, then the reasoning.  
    3. Think carefully and give ONLY ONE final answer.  
    4. Follow the sample examples below and answer in the same format.  

    ### SAMPLE EXAMPLES
    
    ---

    Question: Phong trào cách mạng 1930-1931, phong trào dân chủ 1936-1939 và phong trào giải phóng dân tộc 1939-1945 ở Việt Nam đều khẳng định trong thực tiễn
    A. Vai trò lãnh đạo quần chúng đấu tranh chính trị và vũ trang của chính đảng cách mạng
    B. Tầm quan trọng của công tác xây dựng lực lượng chính trị và căn cứ địa cách mạng
    C. Vai trò xung kích của lực lượng vũ trang trong hỗ trợ quần chúng đấu tranh chính trị
    D. Bước phát triển của cách mạng trên hai địa bàn chiến lược nông thôn và thành thị

    Answer: [A]  
    Reasoning:  
    - All three movements showed the Communist Party of Vietnam’s leadership in both political and armed struggles.  
    - They affirmed the leading role of the revolutionary party in organizing the masses.  
    => Correct answer: A  

    ---

    Question: Hoạt động của Nguyễn Ái Quốc trong những năm 1920-1930 có điểm mới nào so với hoạt động của các sĩ phu tiến bộ Việt Nam đầu thế kỉ XX?
    A. Tập hợp lực lượng cách mạng gồm nhiều tầng lớp trong xã hội
    B. Thành lập Chi bộ Cộng sản đầu tiên của Việt Nam ở Bắc Kì
    C. Trang bị lí luận cách mạng giải phóng dân tộc cho thanh niên
    D. Vận động thanh niên tham gia tổ chức yêu nước chống đế quốc 

    Answer: [C]  
    Reasoning:  
    - Early 20th-century scholars mostly appealed to patriotism and reforms.  
    - Nguyễn Ái Quốc spread Marxism–Leninism and trained youth with revolutionary theory for national liberation.  
    => Correct answer: C  

    ---

    Question: Trong khởi nghĩa từng phần (từ tháng 3 đến tháng 8-1945), nhân dân Việt Nam ở Bắc Kì và Bắc Trung Kì thực hiện khẩu hiệu
    A. "Phá kho thóc, giải quyết nạn đói"
    B. "Người cày có ruộng"
    C. "Tăng gia sản xuất"
    D. "Không một tấc đất bỏ hoang" 

    Answer: [A]  
    Reasoning:  
    - At that time, famine in the North was extremely serious.  
    - Việt Minh launched the slogan "Break open rice stores, fight famine" to mobilize the masses.  
    => Correct answer: A  

    ---

    Question: Trong 5 năm đầu (1986-1990) của công cuộc đổi mới, Việt Nam đạt được một trong những thành tựu nào sau đây?
    A. Hoàn thành hiện đại hóa đất nước
    B. Kiềm chế một bước đà lạm phát
    C. Xuất khẩu gạo đứng đầu thế giới
    D. Hoàn thành công nghiệp hóa đất nước 

    Answer: [B]  
    Reasoning:  
    - The early stage of Đổi mới focused on stabilizing the economy.  
    - A key achievement was curbing inflation, though modernization/industrialization were not yet completed.  
    => Correct answer: B  

    ---

    Question: Trong tháng 12-1993, Hiến pháp Liên bang Nga được ban hành, quy định thể chế
    A. Quân chủ chuyên chế
    B. Quân chủ lập hiến
    C. Tổng thống Liên bang
    D. Cộng hòa xã hội chủ nghĩa

    Answer: [C]  
    Reasoning:  
    - The 1993 Russian Constitution established a federal presidential republic.  
    => Correct answer: C  

    ---

    Answer the following history question: {question}  

    Options:  {choices_list}  

    Answer:""".strip()
    if is_vietnamese_model(model_name):
        instructions = fewshot_cot_vi
    else:
        instructions = fewshot_cot_en
    if is_chat_model(model_name):
        return [{"role":"user", "content":instructions}]
    return instructions
build_fewshot_cot_prompt(model_name, label.loc[0, 'question'], label.loc[0, 'choices'])

[{'role': 'user',
  'content': 'Bạn là một chuyên gia lịch sử Việt Nam có kiến thức sâu rộng. \n    Bạn sẽ được cung cấp câu hỏi lịch sử cấp trung học phổ thông với các lựa chọn của câu hỏi đó.\n    Nếu bạn có định giải thích, hãy trả lời đáp án trước khi giải thích.\n    \n    ### MỤC TIÊU\n    Phân tích câu hỏi, và suy nghĩ từng bước một. Đưa ra MỘT câu trả lời đúng trong bốn lựa chọn được cho và đặt chữ cái của câu trả lời trong ngoặc vuông như sau: `[A]`, `[B]`, `[C]` HOẶC `[D]`. \n\n    ## QUY TẮC BẮT BUỘC.\n    1. **KHÔNG** được lặp lại câu hỏi hoặc các lựa chọn. Quy tắc này phải tuân thủ NGHIÊM NGẶT.\n    2. Nếu bạn có định giải thích, hãy trả lời bằng đáp án bạn chọn trước khi giải thích. \n    3. Suy nghĩ kĩ rồi CHỈ đưa ra MỘT đáp án cuối cùng.\n    4. Đọc ví dụ mẫu ở dưới và trả lời câu hỏi theo format.\n    \n    ### VÍ DỤ MẪU\n    \n    ---\n\n    Câu hỏi: Phong trào cách mạng 1930-1931, phong trào dân chủ 1936-1939 và phong trào giải phóng dân tộc 1939-1945 ở Việt Nam đều 

In [20]:
import ast
def is_chat_model(model_name):
    return any(x in model_name.lower() for x in ["chat", "qwen", "gemma", "llama", "vistral"]) #qwen was here

def is_vietnamese_model(model_name):
    return any(x in model_name.lower() for x in ["vinallama", "vistral", "viet", "vi-"])

def build_prompt(model_name, question, choices):
    """Choose the right prompt format."""
    is_chat = is_chat_model(model_name)
    is_vi = is_vietnamese_model(model_name)

    if is_vi:
        base = prompter(question, choices)
        if is_chat:
            return [{"role":"user", "content":base}]
        else:
            return base
    else:
        base = prompter_en(question, choices)
        if is_chat:
            return [{"role":"user", "content":base}]
        else:
            return base
build_prompt("Qwen3", label.loc[0, 'question'], label.loc[0, 'choices'])

[{'role': 'user',
  'content': "### Answer this history question:\nChiến dịch Biên giới thu-đông (1950) và chiến dịch Điện Biên Phủ (1954) ở Việt Nam có điểm giống nhau nào sau đây?\n### Here are your choices:\nA. Có sự linh hoạt, sáng tạo trong phương án tác chiến.\nB. Đánh bại cuộc tấn công của Pháp vào cơ quan đầu não kháng chiến.\nC. Có ý nghĩa quyết định đến thắng lợi cuối cùng trên mặt trận ngoại giáo.\nD. Sử dụng cách đánh công kiên vào tập đoàn cứ điểm.\n### Choose the correct answer and put it inside square brackets like: [A], [B], [C], or [D]. Let's think step by step.\nDo **NOT** repeat the question or choices. You do NOT need explain your answer. \nIf you are going to explain yourself, answer the question with a letter [A], [B], [C] or [D] first, then explain your answer.\n### Answer: "}]

In [21]:
#7
from tqdm import trange
import re

def extract_letter(text):
    text = text.strip()
    # 1. Prefer square bracket format [A], [B], etc.
    match = re.search(r"\[([A-D])\]", text, re.IGNORECASE)
    if match:
        return match.group(1).upper()
    # 2. Original "### Answer:" format
    match = re.search(r"###\s*Answer[:：]?\s*([A-D])\b", text, re.IGNORECASE)
    if match:
        return match.group(1).upper()
    # 3. Common patterns like "The correct answer is: B"
    match = re.search(r"(?:answer\s*(is|:)?\s*)?[\*\s]*([A-D])[\*\s]*[\.\n]", text, re.IGNORECASE)
    if match:
        return match.group(2).upper()
    # 4. First standalone A-D in first ~100 characters
    match = re.search(r"\b([A-D])\b", text[:100], re.IGNORECASE)
    if match:
        return match.group(1).upper()

    return None


def test_accuracy(model, tokenizer, *, test, model_name, batch_size=8, maxTokens=10):
    total = len(test)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = 'left'
    is_chat = is_chat_model(model_name)
    is_vilm = model_name == "vilm/vinallama-7b-chat"
    
    if is_chat and not is_vilm:
        tokenizer.chat_template = '''
        {% for message in messages %}
        {{ message['content'].strip() + '\n' }}
        {% endfor %}
        {% if add_generation_prompt %}
        {{ '' }}
        {% endif %}
        '''.strip()
        
    test_df = test.copy()
    prompts = []

    for i in range(total):
        question = test_df.loc[i, "question"]
        choices = test_df.loc[i, "choices"]
        prompt = build_prompt(model_name, question, choices) # change prompt here
        if is_vilm:
            content = prompt[0]['content'] if isinstance(prompt, list) else prompt
            text = (
                "<|im_start|>system\n"
                "Bạn là một trợ lí AI hữu ích. Hãy trả lời người dùng một cách chính xác.\n"
                "<|im_end|>\n"
                "<|im_start|>user\n"
                f"{content}<|im_end|>\n"
                "<|im_start|>assistant"
            )
            print(text)
        elif "qwen3" in model_name.lower():
            #print("Qwen3 model detected")
            text = tokenizer.apply_chat_template(
                prompt, tokenize=False, add_generation_prompt=True, enable_thinking=False
            )
        elif is_chat and isinstance(prompt, list):
            text = tokenizer.apply_chat_template(
                prompt, tokenize=False, add_generation_prompt=True
            )
        else:
            text = prompt

        prompts.append(text)

    predictions = []
    raw_responses = []

    if batch_size > total:
        print(f"WARNING: Current batch size {batch_size} is larger than number of questions. Changing to 1 instead.")
        batch_size = 1

    for i in trange(0, total, batch_size):
        batch_prompts = prompts[i:i + batch_size]
        inputs = tokenizer(batch_prompts, return_tensors="pt", padding=True, truncation=True, max_length=2048).to(model.device)

        print("Max token ID:", inputs.input_ids.max().item(), "vs vocab size:", tokenizer.vocab_size)
        if "qwen3" in model_name.lower():
            outputs = model.generate(
                **inputs,
                max_new_tokens=maxTokens,
                do_sample=True,
                top_p=0.8, #0.95 for thinking or 0.8
                top_k=20, #20 for both
                temperature=0.7, #0.6 for thinking or 0.7
                min_p=0.0,
                pad_token_id=tokenizer.eos_token_id
            )
        else: 
            outputs = model.generate(
                **inputs,
                max_new_tokens=maxTokens,
                do_sample=False,
                top_p=None,
                top_k=None,
                temperature=None,
                pad_token_id=tokenizer.eos_token_id
            )

        generated_ids = [output[len(input_ids):] for input_ids, output in zip(inputs.input_ids, outputs)]
        decoded_outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

        print(decoded_outputs)
        raw_responses.extend(decoded_outputs)
        predictions.extend([extract_letter(text) for text in decoded_outputs])
        print(predictions)

    # Save both the full response and extracted answer
    test_df[f"{model_name}_response"] = raw_responses
    test_df[f"{model_name}_answer"] = predictions

    labels = test_df['answer'].str.strip().str.upper()
    correct = sum([p == l for p, l in zip(predictions, labels)])
    accuracy = correct / total

    return accuracy, test_df

In [38]:
#8 - sửa lại VAL và TEST.HEAD và TÊN file
import pandas as pd
from tqdm import trange
import re

def pipeline_cot_en(model_name, test):
    tokenizer, model = load_model(model_name)
    print(model.device)
    for val in [50]:
        accuracy, test_df = test_accuracy(model, tokenizer, test=test, model_name=model_name, batch_size=4, maxTokens=val)
        #print(f"Accuracy on {val} max new tokens: {accuracy}")
        safe_name = model_name.replace("/", "_")
        test_df.to_csv(f"NQ_{safe_name}_cot_on_{val}_maxNewTokens_{accuracy}.csv", index=False)
    return accuracy, test_df
#accuracy, test_df = pipeline_cot_en("vilm/vinallama-2.7b", test)
#accuracy
#test_df

In [28]:
#9 nếu cần clear cuda khi nó hết gpu
import gc
import torch

def clear_cuda():
    gc.collect()                    # Python garbage collection
    if torch.cuda.is_available():
        torch.cuda.empty_cache()    # Releases unreferenced memory back to the pool
        torch.cuda.ipc_collect()    # Cleans up inter-process memory
clear_cuda()

In [17]:
#10
model_name = "Viet-Mistral/Vistral-7B-Chat"
accuracy, ans_df = pipeline_cot_en(model_name, label)
ans_df

Loading model: Viet-Mistral/Vistral-7B-Chat
Loading big boy: Viet-Mistral/Vistral-7B-Chat


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Model successfully loaded across devices:
 - model.embed_tokens.weight on cuda:0
cuda:0


  0%|          | 0/50 [00:00<?, ?it/s]

Max token ID: 38358 vs vocab size: 38365


...

['   Lập luận:\n    - Hội Việt Nam Cách mạng Thanh niên là tổ chức tiền thân của Đảng Cộng sản Việt Nam.\n    - Các hoạt động của Hội tập trung vào việc đào tạo cán bộ cách mạng, chuẩn bị cho sự ra']
[None, None, None, None, None, None, None, None, None, None, None, None, 'A', None, None, None, 'C', None, None, None, 'C', None, 'C', None, 'C', 'C', 'C', 'C', 'A', None, None, None, None, None, 'A', 'C', None, 'C', 'B', None, None, None, 'A', None, 'A', 'A', None, None, None]
Max token ID: 38358 vs vocab size: 38365


100%|██████████| 50/50 [37:29<00:00, 44.99s/it]

['   Lập luận:\n    - Hội Việt Nam Cách mạng Thanh niên là tổ chức tiền thân của Đảng Cộng sản Việt Nam.\n    - Trong giai đoạn 1925-1929, Hội chủ yếu hoạt động tuyên truyền']
[None, None, None, None, None, None, None, None, None, None, None, None, 'A', None, None, None, 'C', None, None, None, 'C', None, 'C', None, 'C', 'C', 'C', 'C', 'A', None, None, None, None, None, 'A', 'C', None, 'C', 'B', None, None, None, 'A', None, 'A', 'A', None, None, None, None]



/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,id,answer,question_type,grade,choices,question,nhan_biet,thong_hieu,van_dung,van_dung_cao,dang_cau_hoi,Viet-Mistral/Vistral-7B-Chat_response,Viet-Mistral/Vistral-7B-Chat_answer
0,history-2022-302-35,A,multiple_choice,12.0,"['Có sự linh hoạt, sáng tạo trong phương án tá...",Chiến dịch Biên giới thu-đông (1950) và chiến ...,NaN,NaN,1.0,NaN,"So sánh, đối chiếu",Lập luận:\n - Chiến dịch Biên giới thu-đ...,None
1,history-2022-304-37,A,multiple_choice,12.0,['Sử dụng đòn tiến công chiến lược của lực lượ...,Chiến dịch Biên giới thu-đông (1950) và chiến ...,NaN,NaN,1.0,NaN,"So sánh, đối chiếu",Lập luận:\n - Chiến dịch Biên giới thu-đ...,None
2,history-2020-304-21,A,multiple_choice,12.0,"['sự bất lực của quân đội Sài Gòn', 'nhiệm vụ ...",Chiến thắng Phước Long (tháng 1-1975) của quân...,NaN,NaN,1.0,NaN,Đánh giá,Lập luận:\n - Chiến thắng Phước Long cho...,None
3,history-2020-301-32,A,multiple_choice,12.0,"['khả năng thắng lớn của quân giải phóng.', 'q...",Chiến thắng Phước Long (tháng 1-1975) của quân...,NaN,NaN,1.0,NaN,Đánh giá,Lập luận:\n - Chiến thắng Phước Long đã ...,None
4,history-2020-302-23,A,multiple_choice,12.0,"['sức mạnh to lớn của quân giải phóng.', 'quân...",Chiến thắng Phước Long (tháng 1-1975) của quân...,NaN,NaN,1.0,NaN,Đánh giá,Lập luận:\n - Chiến thắng Phước Long cho...,None
5,history-2020-303-25,B,multiple_choice,12.0,"['quân đội Sài Gòn đã tan rã hoàn toàn.', 'sự ...",Chiến thắng Phước Long (tháng 1-1975) của quân...,NaN,NaN,1.0,NaN,Đánh giá,Lập luận:\n - Chiến thắng Phước Long đã ...,None
6,history-2020-303-11,D,multiple_choice,12.0,"['làm thất bại chiến tranh cục bộ.', 'làm thất...",Chiến thắng Việt Bắc thu-đông năm 1947 của quâ...,NaN,1.0,NaN,NaN,Kết quả / Ý nghĩa / Vai trò,Lập luận:\n - Chiến thắng Việt Bắc thu-đ...,None
7,history-2020-301-1,D,multiple_choice,12.0,"['làm thất bại chiến tranh tổng lực.', 'làm th...",Chiến thắng Việt Bắc thu-đông năm 1947 của quâ...,NaN,1.0,NaN,NaN,Kết quả / Ý nghĩa / Vai trò,Lập luận:\n - Chiến thắng Việt Bắc thu-đ...,None
8,history-2022-301-34,B,multiple_choice,12.0,['sức mạnh của quân chủ lực khi tấn công vào t...,Chiến thắng Điện Biên Phủ (1954) và thắng lợi ...,NaN,NaN,NaN,1.0,"So sánh, đối chiếu",Lập luận:\n - Điện Biên Phủ (1954) là tr...,None
9,history-2022-303-34,B,multiple_choice,12.0,['giá trị của trận quyết chiến chiến lược đánh...,Chiến thắng Điện Biên Phủ (1954) và thắng lợi ...,NaN,NaN,NaN,1.0,Đánh giá,Lập luận:\n - Chiến thắng Điện Biên Phủ ...,None


In [ ]:
model_list = ["Viet-Mistral/Vistral-7B-Chat"]
#model_list = ["google/gemma-3-1b-it", "google/gemma-1.1-2b-it"]
#model_list = ["Qwen/Qwen3-4B", "Qwen/Qwen2-0.5B-Instruct", "Qwen/Qwen2.5-3B-Instruct", "Qwen/Qwen2.5-7B-Instruct", "google/gemma-3-1b-it", "google/gemma-1.1-2b-it", "google/gemma-7b-it", "vilm/vinallama-2.7b", "vilm/vinallama-7b-chat", "Viet-Mistral/Vistral-7B-Chat"]
#model_list = ["Qwen/Qwen3-4B", "Qwen/Qwen2-0.5B-Instruct", "Qwen/Qwen2.5-3B-Instruct"]
model_list = ["Qwen/Qwen3-8B"]

for m in model_list:
    accuracy, ans_df = pipeline_cot_en(m, label)

Loading model: Qwen/Qwen3-8B


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

...

model-00005-of-00005.safetensors:   0%|          | 0.00/1.24G [00:00<?, ?B/s]

model-00003-of-00005.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/4.00G [00:00<?, ?B/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/3.19G [00:00<?, ?B/s]

In [1]:
ans_df

NameError: name 'ans_df' is not defined

In [ ]:
accuracy

In [23]:
print("Are you sure bro")
a = int(input())
if a == 123:
    !rm -rf /kaggle/working/*
else:
    print("Ok good just making sure")

Are you sure bro


 123


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
for i in ans_df['Qwen/Qwen3-0.6B_response'].head(10):
    print(i)

In [ ]:
for i in ans_df['Qwen/Qwen3-0.6B_response'].head(10):
    print(i)

In [ ]:
ans_df['Qwen/Qwen3-0.6B_response'].head(10)

In [ ]:
ans_df.loc[558, 'Qwen/Qwen3-0.6B_response']

In [ ]:
print(ans_df.loc[0, "Qwen/Qwen3-4B_response"])

In [ ]:
accuracy